In [1]:
import sympy
from sympy import *
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
plt.style.use("seaborn")
import pandas as pd
import cufflinks as cf
import pyfolio as pyf
import plotly.express as px
# from backtesting import Strategy
# from backtesting.lib import crossover
# from backtesting import Backtest
from itertools import product

/Users/miguelbetances/opt/anaconda3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning:

Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.



# Get Data

In [2]:
from backtesting._plotting import plot

/Users/miguelbetances/opt/anaconda3/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

In [3]:
stocks = yf.download('NQ=F', period = '3mo', interval = '60m') #start = '2022-07-13', end = '2023-09-12')
df = stocks
df.to_csv('stocks')

[*********************100%***********************]  1 of 1 completed


In [4]:
stocks.rename(columns = {'Adj Close':'AdjClose'}, inplace = True)

# Arrange Dataframe

In [5]:
short_ema = stocks.AdjClose.ewm(span = 50, min_periods = 50).mean()
#short_sma = stocks.AdjClose.rolling(window=50).mean()
long_sma = stocks.AdjClose.rolling(window=200).mean()
#long_ema = stocks.AdjClose.ewm(span = 200, min_periods = 200).mean()
df['short_ema'] = short_ema
df["long_sma"] = long_sma

In [6]:
difference = (short_ema - long_sma)
df['difference'] = difference
df['difference_ma'] = difference.rolling(window=25).mean()
df['deriv'] = diff(df['difference_ma'])
df['deriv2'] = diff(df['deriv'])
df['roc'] = df['AdjClose'].pct_change(periods=60)*100
df['z'] = df['difference'] * .1

# def stock_calculations(data, column, p1, p2, roc_window):
#     short_ema = data.column.ewm(span = p1, min_periods = p1)
#     long_sma = data.column.rolling(window = p2).mean()
#     data['short_ema'] = short_ema
#     data['long_sma'] = long_sma
#     difference = (short_ema - long_sma)
#     data['difference_ma'] = difference.rolling(window=25).mean()
#     data['deriv2'] = diff(data['deriv'])
#     data['roc'] = data[column].pct_change(periods=roc_window)*100
#     data['z'] = data['difference'] * .1

In [7]:
def stock_calculations(data, col, p1, p2, roc_window):
    short_ema = data.col.ewm(span = p1, min_periods = p1)
    long_sma = data.col.rolling(window = p2).mean()
    data['short_ema'] = short_ema
    data['long_sma'] = long_sma
    difference = (short_ema - long_sma)
    data['difference_ma'] = difference.rolling(window=25).mean()
    data['deriv2'] = diff(data['deriv'])
    data['roc'] = data[column].pct_change(periods=roc_window)*100
    data['z'] = data['difference'] * .1

In [30]:
dev_data = stock_calculations(df, 'AdjClose', 50, 200, 60)

AttributeError: 'DataFrame' object has no attribute 'column'

In [8]:
# fig, ax = plt.subplots(2,1)
# ax[0].plot(derivative_data.index, derivative_data['Close'], color = 'green')
# ax2 = ax.twinx()
# ax2.plot(derivative_data.index, derivative_data['long_sma'], color = 'r', linestyle = '--')
# ax3 = ax2.twinx()
# ax3.plot(derivative_data.index, derivative_data['short_ema'], color = 'b', linestyle = '--')
# ax[1] = 

In [9]:
derivative_data = df.dropna()
derivative_data = derivative_data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis =1)

In [10]:
derivative_data.head()

,AdjClose,short_ema,long_sma,difference,difference_ma,deriv,deriv2,roc,z
Datetime,,,,,,,,,
2023-10-13 08:00:00-04:00,15329.0,15322.700058,15073.22875,249.471308,283.349721,-1.994991,-0.119226,0.287864,24.947131
2023-10-13 09:00:00-04:00,15347.0,15323.653105,15075.00125,248.651855,281.250737,-2.098984,-0.103993,0.389207,24.865186
2023-10-13 10:00:00-04:00,15201.0,15318.842654,15076.05625,242.786404,278.921115,-2.329621,-0.230637,-0.551185,24.278640
2023-10-13 11:00:00-04:00,15119.0,15311.004864,15076.68125,234.323614,276.262959,-2.658156,-0.328535,-1.110294,23.432361
2023-10-13 12:00:00-04:00,15165.0,15305.278605,15077.55000,227.728605,273.284562,-2.978397,-0.320241,-0.856433,22.772860


In [11]:
derivative_data['deriv2'].mean()

-0.0008784893689889831

# Define Strat

In [12]:
# alpha = derivative_data['deriv'].mean() *.1
# alpha2 = derivative_data['deriv2'].mean() 
# count = 0
# position_status = {'Type':None, 'EntryPrice':None, 'Date':None}
# profits = 0
# positions = []
# realized_profits = []
# longs = []
# shorts = []
# pos = []
# for row in derivative_data.iterrows():
#     positions.append(position_status['Type'])
#     realized_profits.append(profits)
#     close_price = row[1][0]
#     long_ma = row[1][2]
#     rate = row[1][7]
#     d1 = row[1][5]
#     d2 = row[1][6]
#     difference_value = row[1][4]
#     position_date = row[0]
#     if d1 < alpha and d2 > 0 and close_price < long_ma and rate > 0 and position_status['Type'] != 'Long':
#         if position_status['Type'] == 'Short':
#             profit = (close_price-position_status['EntryPrice'])*-1
#             profits += profit
#             position_status['Type'] = None
#             position_status['EntryPrice'] = None
#             pos.append(1)
#         if position_status['Type'] == None:
#             pos.append(1)
#             position_status['Type'] = 'Long'
#             position_status['EntryPrice'] = close_price
#             position_status['Date'] = position_date
#     elif d1 > alpha and d2 < 0 and close_price > long_ma and rate < 0 and position_status['Type'] != 'Short':
#         if position_status['Type'] == 'Long':
#             profit = (close_price-position_status['EntryPrice'])
#             profits += profit
#             position_status['Type'] = None
#             position_status['EntryPrice'] = None
#             pos.append(-1)
#         if position_status['Type'] == None:
#             pos.append(-1)
#             position_status['Type'] = 'Short'
#             position_status['EntryPrice'] = close_price
#             position_status['Date'] = position_date
#     elif d2 < (alpha2*-1) and rate < 0:
#         if position_status['Type'] == 'Long':
#             profit = (close_price-position_status['EntryPrice'])
#             profits += profit
#             position_status['Type'] = None
#             position_status['EntryPrice'] = None
#             pos.append(-1)
#     elif d2 > alpha2 and rate > 0:
#         if position_status['Type'] == 'Short':
#             profit = (close_price-position_status['EntryPrice'])*-1
#             profits += profit
#             position_status['Type'] = None
#             position_status['EntryPrice'] = None
#             pos.append(1)
#     count += 1
#     if len(realized_profits) < count:
#         realized_profits.append(0)
#     if len(pos) < count:
#         pos.append(0)
# derivative_data['realized_profits'] = realized_profits
# derivative_data['pos'] = pos
# derivative_data

In [13]:
alpha = derivative_data['deriv'].mean() *.1
alpha2 = derivative_data['deriv2'].mean() 
count = 0
position_status = {'Type':None, 'EntryPrice':None, 'Date':None}
condition_met = {'D1': None, 'D2': None, 'ClosePrice': None, 'Rate': None}
profits = 0
positions = []
realized_profits = []
longs = []
shorts = []
pos = []
for row in derivative_data.iterrows():
    positions.append(position_status['Type'])
    realized_profits.append(profits)
    close_price = row[1][0]
    long_ma = row[1][2]
    rate = row[1][7]
    d1 = row[1][5]
    d2 = row[1][6]
    difference_value = row[1][4]
    position_date = row[0]
    if d1 < alpha and d2 > 0 and close_price < long_ma and rate > 0 and position_status['Type'] != 'Long':
        if position_status['Type'] == 'Short':
            profit = (close_price-position_status['EntryPrice'])*-1
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(1)
        if position_status['Type'] == None:
            pos.append(1)
            position_status['Type'] = 'Long'
            position_status['EntryPrice'] = close_price
            position_status['Date'] = position_date
    elif d1 > alpha and d2 < 0 and close_price > long_ma and rate < 0 and position_status['Type'] != 'Short':
        if position_status['Type'] == 'Long':
            profit = (close_price-position_status['EntryPrice'])
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(-1)
        if position_status['Type'] == None:
            pos.append(-1)
            position_status['Type'] = 'Short'
            position_status['EntryPrice'] = close_price
            position_status['Date'] = position_date
    elif d2 < (alpha2*-1) and rate < 0:
        if position_status['Type'] == 'Long':
            profit = (close_price-position_status['EntryPrice'])
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(-1)
    elif d2 > alpha2 and rate > 0:
        if position_status['Type'] == 'Short':
            profit = (close_price-position_status['EntryPrice'])*-1
            profits += profit
            position_status['Type'] = None
            position_status['EntryPrice'] = None
            pos.append(1)
    count += 1
    if len(realized_profits) < count:
        realized_profits.append(0)
    if len(pos) < count:
        pos.append(0)
derivative_data['realized_profits'] = realized_profits
derivative_data['pos'] = pos

In [14]:
values = derivative_data[derivative_data['pos'] == 1]
print(len(values))

3


# Marking entry and exit

In [15]:
entry = [derivative_data.loc[(derivative_data.pos == 1)]][0]
exit = [derivative_data.loc[(derivative_data.pos == -1)]][0]

In [16]:
exit['exit'] = exit['AdjClose']
exit = exit.drop('AdjClose', axis = 1)

In [17]:
entry['entry'] = entry['AdjClose']
entry= entry.drop('AdjClose', axis = 1)

In [18]:
entry = pd.DataFrame(index = entry.index, data = {'entry':entry.entry})

In [19]:
len(entry)

3

In [20]:
exit = pd.DataFrame(index = exit.index, data = {'exit':exit.exit})

In [21]:
len(exit)

3

In [22]:
derivative_data = derivative_data.join(entry)

In [23]:
derivative_data = derivative_data.join(exit)

In [24]:
buy_hold_return = (derivative_data.AdjClose - derivative_data.AdjClose.shift(1)).cumsum()
derivative_data['bnh'] = buy_hold_return

In [25]:
derivative_data

,AdjClose,short_ema,long_sma,difference,difference_ma,deriv,deriv2,roc,z,realized_profits,pos,entry,exit,bnh
Datetime,,,,,,,,,,,,,,
2023-10-13 08:00:00-04:00,15329.00,15322.700058,15073.22875,249.471308,283.349721,-1.994991,-0.119226,0.287864,24.947131,0.00,0,NaN,NaN,NaN
2023-10-13 09:00:00-04:00,15347.00,15323.653105,15075.00125,248.651855,281.250737,-2.098984,-0.103993,0.389207,24.865186,0.00,0,NaN,NaN,18.00
2023-10-13 10:00:00-04:00,15201.00,15318.842654,15076.05625,242.786404,278.921115,-2.329621,-0.230637,-0.551185,24.278640,0.00,0,NaN,NaN,-128.00
2023-10-13 11:00:00-04:00,15119.00,15311.004864,15076.68125,234.323614,276.262959,-2.658156,-0.328535,-1.110294,23.432361,0.00,0,NaN,NaN,-210.00
2023-10-13 12:00:00-04:00,15165.00,15305.278605,15077.55000,227.728605,273.284562,-2.978397,-0.320241,-0.856433,22.772860,0.00,0,NaN,NaN,-164.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 10:00:00-05:00,17019.50,17097.676918,16996.08500,101.591918,133.287078,-2.515205,-0.142443,-0.370257,10.159192,885.75,0,NaN,NaN,1690.50
2023-12-29 11:00:00-05:00,16976.75,17092.934686,16996.84125,96.093436,130.610927,-2.676150,-0.160945,-0.603053,9.609344,885.75,0,NaN,NaN,1647.75
2023-12-29 12:00:00-05:00,16994.25,17089.064698,16997.67000,91.394698,127.807262,-2.803666,-0.127516,-0.475828,9.139470,885.75,0,NaN,NaN,1665.25


# Plots

In [26]:
fig = px.line(derivative_data, x=derivative_data.index, y=['AdjClose','short_ema', 'long_sma','difference'])
fig.add_scatter(x=derivative_data.index, y=derivative_data.entry, mode="markers", marker=dict(size=8,color="yellow"), name = 'Entry')
fig.add_scatter(x=derivative_data.index, y=derivative_data.exit, mode="markers", marker=dict(size=8,color="Red"), name = 'Exit')
fig.show()

In [27]:
fig2 = px.line(derivative_data, x=derivative_data.index, y=['z', 'deriv', 'deriv2', 'roc'])
fig2.show()

In [28]:
derivative_data[['bnh', 'realized_profits']].iplot()

In [29]:
derivative_data

,AdjClose,short_ema,long_sma,difference,difference_ma,deriv,deriv2,roc,z,realized_profits,pos,entry,exit,bnh
Datetime,,,,,,,,,,,,,,
2023-10-13 08:00:00-04:00,15329.00,15322.700058,15073.22875,249.471308,283.349721,-1.994991,-0.119226,0.287864,24.947131,0.00,0,NaN,NaN,NaN
2023-10-13 09:00:00-04:00,15347.00,15323.653105,15075.00125,248.651855,281.250737,-2.098984,-0.103993,0.389207,24.865186,0.00,0,NaN,NaN,18.00
2023-10-13 10:00:00-04:00,15201.00,15318.842654,15076.05625,242.786404,278.921115,-2.329621,-0.230637,-0.551185,24.278640,0.00,0,NaN,NaN,-128.00
2023-10-13 11:00:00-04:00,15119.00,15311.004864,15076.68125,234.323614,276.262959,-2.658156,-0.328535,-1.110294,23.432361,0.00,0,NaN,NaN,-210.00
2023-10-13 12:00:00-04:00,15165.00,15305.278605,15077.55000,227.728605,273.284562,-2.978397,-0.320241,-0.856433,22.772860,0.00,0,NaN,NaN,-164.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 10:00:00-05:00,17019.50,17097.676918,16996.08500,101.591918,133.287078,-2.515205,-0.142443,-0.370257,10.159192,885.75,0,NaN,NaN,1690.50
2023-12-29 11:00:00-05:00,16976.75,17092.934686,16996.84125,96.093436,130.610927,-2.676150,-0.160945,-0.603053,9.609344,885.75,0,NaN,NaN,1647.75
2023-12-29 12:00:00-05:00,16994.25,17089.064698,16997.67000,91.394698,127.807262,-2.803666,-0.127516,-0.475828,9.139470,885.75,0,NaN,NaN,1665.25
